# Encoding machine data (log) to vector DB using Milvus Push

Example Usage: | search ... | fit MLTKContainer algo=llm_rag_log_encoder collection_name=log_events_example embedder_dimension=384 embedder_name="all-MiniLM-L6-v2" use_local=1 label_field_name=_raw _raw search_name src dest into app:llm_rag_log_encoder

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [18]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
import os
import time
import pymilvus
from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"
MILVUS_ENDPOINT = "http://milvus-standalone:19530"

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
print("pymilvus version: " + pymilvus.__version__)

In [19]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [20]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("llm_rag_log_encoder")

## Stage 2 - create and initialize a model

In [14]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}   
    pk_type=DataType.VARCHAR        
    embedding_type=DataType.FLOAT_VECTOR
    # Dimensionality setting of collection
    try:
        embedder_name = param['options']['params']['embedder_name'].strip('\"')
    except:
        embedder_name = 'all-MiniLM-L6-v2'
    # Dimension checking for default embedders
    if embedder_name == 'intfloat/multilingual-e5-large':
        n_dims = 1024
    elif embedder_name == 'all-MiniLM-L6-v2':
        n_dims = 384
    else:
        try:
            n_dims=int(param['options']['params']['embedder_dimension'])
        except:
            n_dims=384
    
    
    # Collection name setting   
    try:
        collection_name=param['options']['params']['collection_name'].strip('\"')
    except:
        collection_name="default_collection"
    # Schema setting
    try:
        schema_fields=df.columns.tolist()
        schema_fields.remove(param['options']['params']['label_field_name'])
    except:
        schema_fields=[]
        
    print("start connecting to Milvus")
    try:
        # this hostname may need changing to a specific local docker network ip address depending on docker configuration
        connections.connect("default", host="milvus-standalone", port="19530")
        collection_exists = utility.has_collection(collection_name)
        
        # Basic schema setting
        fields = [
            FieldSchema(name="_key", is_primary=True, auto_id=True, dtype=DataType.INT64),
            FieldSchema(name="embeddings", dtype=embedding_type, dim=n_dims),
            FieldSchema(name="label", dtype=DataType.VARCHAR, max_length=5000),
        ]
        # Additional schema setting
        if len(schema_fields) != 0: 
            for i in range(len(schema_fields)):
                fields.append(FieldSchema(name=schema_fields[i], dtype=DataType.VARCHAR, max_length=1000))
        # Create schema
        
        schema = CollectionSchema(fields, f"dsdl schema for {collection_name}")
        print(fields)
        
        if collection_exists:
            print(f"The collection {collection_name} already exists")
            collection = Collection(collection_name)
            collection.load()
        else:
            print(f"The collection {collection_name} does not exist")
            print(f"creating new collection: {collection_name}")
            collection = Collection(collection_name, schema, consistency_level="Strong")
            index = {
                "index_type": "IVF_FLAT",
                "metric_type": "L2",
                "params": {"nlist": 1024},
            }
            collection.create_index("embeddings", index)
    except:
        collection = None
    
    model['collection']=collection
    model['collection_name']=collection_name

    return model

In [15]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
print(model)

start connecting to Milvus
[{'name': '_key', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'embeddings', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 1024}}, {'name': 'label', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 5000}}, {'name': 'category', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 1000}}]
The collection tester_cat does not exist
creating new collection: tester_cat
{'collection': <Collection>:
-------------
<name>: tester_cat
<description>: dsdl schema for tester_cat
<schema>: {'auto_id': True, 'description': 'dsdl schema for tester_cat', 'fields': [{'name': '_key', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'embeddings', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 1024}}, {'name': 'label', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'m

## Stage 3 - fit the model

In [18]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    
    return df

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

## Stage 4 - apply the model

In [16]:
# apply your model
# returns the calculated results
def apply(model,df,param):
    if model['collection'] is not None:
        use_local= int(param['options']['params']['use_local'])
        try:
            embedder_name = param['options']['params']['embedder_name'].strip('\"')
        except:
            embedder_name = 'all-MiniLM-L6-v2'
        if use_local:
            embedder_name = f'/srv/app/model/data/{embedder_name}'
            print("Using local embedding model checkpoints")  
        transformer_embedder = HuggingFaceEmbedding(model_name=embedder_name)

        try:
            df=df.copy()
            label_field_name=param['options']['params']['label_field_name']
            label_column = df[label_field_name].astype(str)
        
            text_column = label_column.tolist()
            vector_column = []
            for text in text_column:
                vector_column.append(transformer_embedder.get_text_embedding(text))
            data=[vector_column, label_column.tolist()]
        except:
            data = None
            m = "Failed. Could not vectorize dataframe. Check your field name."
            
        try:
            schema_fields=df.columns.tolist()
            schema_fields.remove(label_field_name)
        except:
            schema_fields=[]
        if data is not None:
            if len(schema_fields) != 0:
                for i in range(len(schema_fields)):  
                    data.append(df[schema_fields[i]].astype(str).tolist())
            try:
                model['collection'].insert(data)
                m = "Success"
            except:
                m = "Failed. Could not insert data to collection."
    else:
        m = "Failed. Could not create collection. Check collection naming."
    df['message'] = [m]*df.shape[0]
    return df['message']

In [17]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param))

/usr/local/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


0    Success
1    Success
2    Success
3    Success
4    Success
5    Success
6    Success
7    Success
8    Success
9    Success
Name: message, dtype: object


## Stage 5 - save the model

In [1]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    return model

## Stage 6 - load the model

In [2]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    return model

## Stage 7 - provide a summary of the model

In [17]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

After implementing your fit, apply, save and load you can train your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer algo=barebone s from feature_* into app:barebone_model<br>

Or apply your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| apply barebone_model as the_meaning_of_life

## Send data back to Splunk HEC
When you configured the Splunk HEC Settings in the DSDL app you can easily send back data to an index with [Splunk's HTTP Event Collector (HEC)](https://docs.splunk.com/Documentation/Splunk/latest/Data/UsetheHTTPEventCollector). Read more about data formats and options in the [documentation](https://docs.splunk.com/Documentation/Splunk/latest/Data/FormateventsforHTTPEventCollector#Event_metadata).

### Use cases
- you want to offload longer running, possibly distributed computations that need to deliver results asynchroneously back into Splunk. 
- you might not want to present results back into the search pipeline after your `| fit` or `| apply` command. 
- you can easily utilize this approach for any logging purposes or other profiling tasks in your ML code so you can actively monitor and analyze your processes.

### Example

In [18]:
from dsdlsupport import SplunkHEC as SplunkHEC
hec = SplunkHEC.SplunkHEC()

In [19]:
# example to send 10 hello world events
response = hec.send_hello_world(10)

In [20]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

HEC endpoint http://host.docker.internal:8088/services/collector/event 
returned with status code 200 
and response message: {"text":"Success","code":0}


In [21]:
# example to send a JSON object, e.g. to log some data
from datetime import datetime
response = hec.send({'event': {'message': 'operation done', 'log_level': 'INFO' }, 'time': datetime.now().timestamp()})

In [22]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

HEC endpoint http://host.docker.internal:8088/services/collector/event 
returned with status code 200 
and response message: {"text":"Success","code":0}


## End of Stages
All subsequent cells are not tagged and can be used for further freeform code